In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from shapely.geometry import Point, Polygon
from collections import Counter
import geopandas
from geopy.geocoders import Nominatim
import warnings
import json
import requests
import time
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'shapely'

In [ ]:
df = pd.read_csv('data/complete.csv')
# Controllo come vengono indicizzate le variabili
df.info()

# Analisi argomentativa
Quali sono le keywords piú frequenti?
1) Keywords totali piú frequenti

2) Frequenza annuale di quelle frequenti

3) Prendere in esempio le keywords di riferimento sulla mappa dei fatti di trento, guardare se hanno pattern

In [ ]:
dictionary={
    "Argomenti:" : "",
    "furto" : "furti",
    "orso" : "orsi",
    "trasporto" : "trasporti",
}

for key in dictionary.keys():
    df["keywords"] = df["keywords"].str.replace(key, dictionary[key])
df["keywords"] = df["keywords"].apply(lambda x: re.findall('\'([^\']*)\'', str(x))) # Transform string into list

In [ ]:
def uniqueKeyword(listaInDataframe):
    #df_temp = df[df['year']=='2019']
    temp_l = list(listaInDataframe)
    temp_l = [j for i in temp_l for j in i]
    cnt = Counter(temp_l)

    key_value_count = cnt.most_common()
    key_value_count.sort(key= lambda x: x[1], reverse=True)
    total_keywords = [[x,y] for x,y in key_value_count]
    #print(len(total_keywords))
    return total_keywords

totalKey = uniqueKeyword(df["keywords"])



In [ ]:
pd.DataFrame(totalKey[1:20], columns=['key', 'count']).to_csv("../newSite/src/best50key.csv", index=False)

In [ ]:
# Somma totale di tutte le keyword uniche 
len(totalKey)

In [ ]:
# Studio la variazione delle keywords per ogni anno. Quali sono i trend topic annuali?
years = df.year.unique()

for year in years:
    df_year = df[df['year']==year]
    keywords = list(uniqueKeyword(df_year['keywords']))[:21]
    keywds = []
    for key in keywords:
        if(key[0] != ''):
            keywds.append(key)
    print(year)
    print(keywds)

# Analisi Temporale
Frequenza degli articoli annuali, quanti articoli escono all'anno? qual è la crescita? 

* Quali sono i mesi piú caldi dal punto di vista dell'informazione? Anni?

Lo studio ha dimostrato che per quanto riguarda gli anni non c'è una sostanziale crescita o decrescita dal punto di vista della pubblicazione degli articoli. In genere parliamo di una media di 3000 articoli l'anno che possono variare nell'ordine delle centinaia. I dati del 2020 non sono completi poiché si fermano ad inizio Novembre ma vedendo l'andamento degli anni precedenti si puó stimare una pubblicazione di altri ______ articoli.
L'analisi che segue mostra l'andamento delle pubblicazioni in ordine temporale. La prima immagine descrive l'analisi annuale e la seconda l'andamento mensile nel corso di 8 anni di pubblicazioni.
A spiccare sono proprio alcuni mesi che vedono un picco di crescita nell'ordine delle centinaia di articoli. I mesi in questione sono spesso quelli di ritorno dalle vacanze estive come Settembre ed Ottobre. 
Il 2020 é un anno particolare da questo punto di vista, complice la pandemia. Infatti vediamo come nel periodo di Marzo, Aprile le pubblicazioni siano state piú alte della norma, per non parlare di una fortissima diminuzione ad Agosto, riscontrata ogni anno ma accentuata in questo specifico periodo, ed una ripresa impressionante al ritorno delle vacanze che ha portato ad un divario tra agosto e ottobre di circa 150 articoli in piú.

Analizzando quelli che sono gli argomenti principali dei vari mesi ovviamente il coronavirus é il trend topic di questo 2020. 
Ricordarsi che il grafico deve avere per ogni punto anche il topic principale del mese.


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
# Documentazione groupby per le frequenze delle date
year_frequency = df.groupby(pd.Grouper(key='timestamp', freq='1Y')).size().reset_index(name='counts')
month_frequency = df.groupby(pd.Grouper(key='timestamp', freq='1M')).size().reset_index(name='counts')

In [ ]:
year_frequency['timestamp'] = year_frequency['timestamp'].apply(lambda x: x.strftime("%Y"))
year_frequency.to_csv("../newSite/src/countPerYear.csv", index=False)

In [ ]:
month_frequency['timestamp'] = month_frequency['timestamp'].apply(lambda x: x.strftime("%Y-%m"))

month_frequency.to_csv("../newSite/src/countPerMonth.csv", index=False)
month_frequency

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(30,10), dpi=80, facecolor='w', edgecolor='k')

y = year_frequency['timestamp'].to_list()
c = year_frequency['counts'].to_list()
plt.plot(y,c)
plt.xticks(y)
plt.gcf().autofmt_xdate()
plt.grid(True)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(50,10), dpi=100, facecolor='w', edgecolor='k')

y = month_frequency['timestamp'].to_list()
c = month_frequency['counts'].to_list()
ax = plt.plot(y,c)
plt.xticks(y)
plt.gcf().autofmt_xdate()
plt.grid(True)

Lo studio porta a pensare che ci sono mesi che hanno dei picchi particolari, come Settembre ed ottobre. Controllo i top 15 mesi per frequenza e trovo 8 su 15 sono settembre ed ottobre rispetto agli altri mesi. Ma cosa pubblicano?

In [ ]:
display(month_frequency.sort_values('counts', ascending=True).head(15))

# Come ha cambiato gli argomenti il coronavirus?

In [ ]:
duemila19 = df.loc[(df['timestamp'].dt.year == 2019)]
listaKey2019 = uniqueKeyword(duemila19["keywords"])
duemila20 = df.loc[(df['timestamp'].dt.year == 2020)]
listaKey2020 = uniqueKeyword(duemila20["keywords"])

In [ ]:
argomenti2anni = pd.DataFrame(zip(listaKey2019,listaKey2020), columns=[2019,2020])

In [ ]:
print(argomenti2anni.head(20).to_string())

### Secondo approccio
Comparo i best key del 2019 e vedo che cambiamento in fatto di numeri hanno avuto nel 2020

In [ ]:
years = [2019, 2020]

df_year = df[df['year']==2019]
keywords2019 = list(uniqueKeyword(df_year['keywords']))[:21]
keys = []
key2019 = []
for key in keywords2019:
    if(key[0] != ''):
        keys.append(key[0])
        key2019.append(key)
key2019 = dict(key2019)
key2019

In [ ]:
df_year = df[df['year']==2020]
keywords2020 = list(uniqueKeyword(df_year['keywords']))
key2020 =[]
for key in keywords2020:
    if((key[0] != '') and (key[0] in keys) ):
        key2020.append(key)
key2020 = dict(key2020)
key2020

In [ ]:
cols = ['keys', 'diciannove', 'venti']
ordered2020 = []
for key in keys:
    ordered2020.append(key2020.get(key))
pd.DataFrame(list(zip(keys,key2019.values() ,ordered2020 )), columns=cols).to_csv("../newSite/src/compared20192020.csv", index=False)

# Analisi mensile top Keyword
Abbiamo visto negli ultimi anni Ottobre e le sue argomentazioni, ma quali sono le keywords che hanno caratterizzato ogni mese nel corso degli anni?

Per poterlo realizzare posso utilizzare le funzioni che ho giá creato e ciclarle per creare quello che mi serve

In [ ]:
rows = []
for year in range(2012,2021):
    
    # ciclo sui mesi
    for month in range(1,13):
        data = df.loc[(df['timestamp'].dt.month == month)&(df['timestamp'].dt.year == year)]
        lista = uniqueKeyword(data["keywords"])
        #print(year, month, lista[:3])
        rows.append(lista[:20])

In [ ]:
arrays = [np.array([2012]*12 +[2013]*12 +[2014]*12 +[2015]*12 +[2016]*12 +[2017]*12 +[2018]*12 +[2019]*12 +[2020]*12), 
          np.array([1,2,3,4,5,6,7,8,9,10,11,12]*9)]

In [ ]:
dato = pd.DataFrame(rows, index=arrays, columns=range(1,21))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(dato)

# Analisi Geografica
L'analisi geografica qui rappresentata prende come riferimento il dataset geolocalizzato e lo studia per quelle che possno essere le componenti geografiche e le componenti argomentative.

Qui rispondiamo a domande come:
* Quali sono i topic piú trattati per le notizie geolocalizzate? In questo modo possiamo vedere che tipo di notizie andiamo a trattare. Mi aspetto che siano notizie prevalentemente di incidenti, strade, notizie di quartiere.
* Tra queste notizie quali sono le piú comuni su trento?
* Quali sono i quartieri di trento con piú notizie?
* C'è una sostanziale differenza tra il centro e la periferia?
* Trento VS Comuni. Quali sono le notizie e che tipo di notizie sono quelle nei comuni?
* Occhio mirato sulla criminalità

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
from shapely.geometry import Point, Polygon
from collections import Counter
import geopandas
from geopy.geocoders import Nominatim
import warnings
import json
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/complete.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.info()

In [ ]:
len(df[(df['comune'].notnull())])

I punti trovati in trentino sono 19.216.

Quali sono i topic piú trattati per le notizie geolocalizzate? In questo modo possiamo vedere che tipo di notizie andiamo a trattare. Mi aspetto che siano notizie prevalentemente di incidenti, strade, notizie di quartiere.

In [ ]:
dictionary={
    "Argomenti:" : "",
    "furto" : "furti",
    "orso" : "orsi",
}

for key in dictionary.keys():
    df["keywords"] = df["keywords"].str.replace(key, dictionary[key])
df["keywords"] = df["keywords"].apply(lambda x: re.findall('\'([^\']*)\'', str(x))) # Transform string into list

In [ ]:
def uniqueKeyword(listaInDataframe):
    #df_temp = df[df['year']=='2019']
    temp_l = list(listaInDataframe)
    temp_l = [j for i in temp_l for j in i]
    cnt = Counter(temp_l)

    key_value_count = cnt.most_common()
    key_value_count.sort(key= lambda x: x[1], reverse=True)
    total_keywords = [[x,y] for x,y in key_value_count]
    #print(len(total_keywords))
    return total_keywords

trento = df[df['comune'] == 'Trento' ]
trentino = df[(df['comune'].notnull())]
nonTrento = df[(df['comune'].notnull())&(df['comune']!="Trento")]
totalKey = uniqueKeyword(df["keywords"])
nonTrentoKey = uniqueKeyword(nonTrento["keywords"])
trentoKey = uniqueKeyword(trento["keywords"])


In [ ]:
print("Numero di punti in trentino: ",len(trentino))
print("Numero di punti nei comuni: ",len(nonTrento))
print("Keywors più utilizzate nei comuni: \n", nonTrentoKey[:50])

In [ ]:
nonTrentoKey
keywds = []
for key in nonTrentoKey:
    if(key[0] != ''):
        keywds.append(key)
pd.DataFrame(keywds[:20], columns=['key', 'count']).to_csv("/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/comuni20key.csv", index=False)

In [ ]:
print("Numero di punti a Trento: ",len(trento))
print("Keywors più utilizzate a Trento: \n",trentoKey[:50])

In [ ]:
print("Percentuale di notizie su trento")
print((11343/19216)*100), 

Questo ci fa capire che le notizie sul coronavirus sono date prevalentemente su Trento, considerando che piú della metá sono geolocalizzate in cittá e solo 69 sono rivolte ai comuni.

Come era prevedibile le notizie principali per i comuni sono gli incidenti, quello che stupisce é che le notizie sui furti siano maggiori nei comuni che in cittá. In realtá non so bene come descrivere quello che ho trovato in questo contesto. Magari é utile contestualizzare il tutto con una mappa che definisce i comuni piú pericolosi magari. Faccio il dataset e vedo.

In [ ]:
countArticleComune = nonTrento.groupby(['comune']).size().reset_index(name='counts').sort_values('counts', ascending=False)

Controllo se i comuni che hanno piú articoli sono i piú grandi del Trentino. La risposta é no. Le cittá con piú articoli sono quelle con popolazione maggiore in ordine di densitá di popolazione!!
https://www.tuttitalia.it/trentino-alto-adige/provincia-autonoma-di-trento/88-comuni/popolazione/

In [ ]:
import fiona
import geopandas as gpd
fiona.listlayers('data/istat_administrative_units_2020.gpkg')
municipalities = gpd.read_file("data/istat_administrative_units_2020.gpkg",
                               layer="municipalities", 
                               GEOM_POSSIBLE_NAMES="geometry",
                               KEEP_GEOM_COLUMNS="NO")
# filter the province
municipalities_province_trento = municipalities[municipalities.COD_PROV==22]
municipalities_province_trento

In [ ]:
municipalities_province_trento.to_crs("EPSG:4326", inplace=True)
municipalities_province_trento

In [ ]:
municipalities_province_trento['areas'] = municipalities_province_trento['geometry'].area/10**6

In [ ]:
municipalities_province_trento = municipalities_province_trento[['COMUNE', 'areas']].sort_values('areas', ascending=False)
municipalities_province_trento = municipalities_province_trento.rename(columns={'COMUNE': 'comune'})

In [ ]:
print(pd.merge(countArticleComune, municipalities_province_trento, how='outer', on=['comune']).to_string())

Controllo quali sono i comuni che hanno piú criminalitá in base alle key che ho trovato:
* Furti
* Incidenti

L'analisi dei furti nei comuni,divisi per anni fanno capire che sono notizie sparse, non di particolare rilevanza, certo il numero dei comuni li fa spiccare rispetto a Trento, in realtá dipende dal numero.
Rovereto é quella che tra tutti è la piú rilevante, nulla di particolarmente rilevante rispetto a trento, un massimo di 10 notizie annuali.
Queste analisi stanno portando alla conclusione che la principale fonte di notizie sia Trento e non il Trentino.

# Conto gli articoli nei comuni e li trasformo in geojson

In [ ]:
countArticleTrentino = trentino.groupby(['comune']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
municipalities_province_trento['geometry'] = municipalities_province_trento['geometry'].apply(lambda x : shapely.geometry.MultiPolygon(x)) 
municipalities_province_trento

In [ ]:
df_counts_comuni = municipalities_province_trento.merge(countArticleTrentino, how='left',left_on=['COMUNE'],right_on=['comune'])

In [ ]:
df_counts_comuni = df_counts_comuni[['comune', 'counts', 'geometry']]

In [ ]:
import shapely.geometry
import geopandas as gpd
df_counts_comuni['geometry'] = df_counts_comuni['geometry'].apply(lambda x : shapely.geometry.MultiPolygon(x)) 

In [ ]:
df_counts_comuni.to_file("/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/countComuni.geojson", driver='GeoJSON')

In [ ]:
df_counts_comuni.plot()

# Analisi Trento

* Qual è il quartiere con piú notizie? Che tipo di notizie ci sono?

Un problema che ho notato é che il dataset Trento pinga le notizie in centro storico anche quando é solo "Trento" come geolocalizzazione. 
Vedo di togliere quelle che hanno address solo Trento. Facendolo perdo 1000 punti ma quello che ne ricavo é la geolocalizzazione di solo punti che hanno address centro storico.
Ora posso fare l'analisi.

In [ ]:
trento

In [ ]:
# Tolgo gli address che contengono solo Trento perché vengono pingati in =centro storico
trento = trento[trento['address']!="Trento"]
trentoKey

In [ ]:
conteggioQuartieriDf = trento.groupby(['nome_quart', 'cod_quart']).size().reset_index(name='counts').sort_values('counts', ascending=False)
conteggioQuartieriDf['cod_quart'] = conteggioQuartieriDf['cod_quart'].apply(lambda x : int(x))

In [ ]:
conteggioQuartieriDf

In [ ]:
from shapely import wkt
import shapely.geometry
import geopandas as gpd
#df_quartieri = pd.read_csv('data/poli_sociali.csv', delimiter=';')
#df_quartieri = geopandas.read_file('data/poli_sociali.csv')
# loading polygons geodataframe
gdf_polygons = gpd.read_file('data/poli_sociali/poli_sociali.shp')
gdf_polygons.to_crs("EPSG:4326", inplace=True)
#gdf_polygons

In [ ]:
counts_df = gdf_polygons.merge(conteggioQuartieriDf, how='inner',left_on=['cod_quart', 'nome_quart'],right_on=['cod_quart', 'nome_quart'])
counts_df

In [ ]:
counts_df = counts_df[['counts', 'cod_quart','nome_quart', 'geometry']]

In [ ]:
counts_df['geometry'] = counts_df['geometry'].apply(lambda x : shapely.geometry.Polygon(x)) 
type(counts_df)

## Transformo i dati in json

#counts_df.to_csv('./data/countsComuni.csv', index=False)
#geojson = counts_df.to_json()
counts_df.to_file("/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/countQuartieri.geojson", driver='GeoJSON')
#counts_df.to_geojson()

with open('/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/countQuartieri.json', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=True)

## Analizzo quali sono le principali notizie per tipo di quartiere

In [ ]:
quartieri = trento['nome_quart'].unique()
quartieri = [i for i in quartieri]
print(type(quartieri))
print(len(np.array(quartieri*9)))

In [ ]:
len(np.array([2012]*len(quartieri) +[2013]*len(quartieri) +[2014]*len(quartieri) +[2015]*len(quartieri) +[2016]*len(quartieri) +[2017]*len(quartieri) +[2018]*len(quartieri) +[2019]*len(quartieri) +[2020]*len(quartieri)))

In [ ]:
rows = []

for quartiere in quartieri:
    data = trento.loc[trento['nome_quart'] == str(quartiere)]
    lista = uniqueKeyword(data["keywords"])
    #print(year, month, lista[:3])
    rows.append(lista[:20])

In [ ]:
arrays = [np.array(quartieri)]

In [ ]:
dato = pd.DataFrame(rows, index=arrays, columns=range(1,21))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(dato)

In [ ]:
# Dove sono collocate le notizie di furti e droga?
trento[trento['keywords'].apply(lambda x: 'droga' in x)].groupby(['nome_quart']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
# Il centro storico é quello piú rappresentativo della cittá 
centro = trento[trento['nome_quart']== "Centro Storico".upper()]

In [ ]:
print(centro[centro['keywords'].apply(lambda x: 'droga' in x)].groupby(pd.Grouper(key='timestamp', freq = '1Y')).size().reset_index(name='counts').to_string())

In [ ]:
print(centro[(centro['keywords'].apply(lambda x: 'droga' in x))&(centro['year'] == 2016)]['title'].to_list())

In [ ]:
trento[trento['keywords'].apply(lambda x: 'furti' in x)].groupby(['nome_quart']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
print(trento[trento['keywords'].apply(lambda x: 'droga' in x)].groupby(['nome_quart']).size().to_string())#.sort_values('counts', ascending=False)

In [ ]:
print(centro[centro['keywords'].apply(lambda x: 'furti' in x)].groupby(pd.Grouper(key='timestamp', freq = '1Y')).size().reset_index(name='counts').to_string())

In [ ]:
trento[trento['keywords'].apply(lambda x: 'droga' in x)].groupby(pd.Grouper(key='timestamp', freq = '1Y')).size().reset_index(name='counts').to_csv("/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/drogaTrento.csv", index=False)

In [ ]:
trento[trento['keywords'].apply(lambda x: 'furti' in x)].groupby(pd.Grouper(key='timestamp', freq = '1Y')).size().reset_index(name='counts').to_csv("/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/newSite/src/furtiTrento.csv", index=False)

In [ ]:
print(centro[(centro['keywords'].apply(lambda x: 'furti' in x))&(centro['year'] == 2016)]['title'].to_list())

# Analisi Argomentativa
In questa analisi studio le co-occorrenze tra le keywords per capire se in qualche modo alcuni argomenti possano essere collegati e trovare qualche relazione in particolare. 

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
from shapely.geometry import Point, Polygon
from collections import Counter
import geopandas
from geopy.geocoders import Nominatim
import warnings
import json
import requests
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv('data/complete.csv')
# Controllo come vengono indicizzate le variabili
df.info()

In [ ]:
dictionary={
    "Argomenti:" : "",
    "furto" : "furti",
    "orso" : "orsi",
}

for key in dictionary.keys():
    df["keywords"] = df["keywords"].str.replace(key, dictionary[key])
df["keywords"] = df["keywords"].apply(lambda x: re.findall('\'([^\']*)\'', str(x))) # Transform string into list

In [ ]:
#Check unique keywords
def uniqueKeyword(listaInDataframe):
    #df_temp = df[df['year']=='2019']
    temp_l = list(listaInDataframe)
    temp_l = [j for i in temp_l for j in i]
    cnt = Counter(temp_l)

    key_value_count = cnt.most_common()
    key_value_count.sort(key= lambda x: x[1], reverse=True)
    total_keywords = [[x,y] for x,y in key_value_count]
    #print(len(total_keywords))
    return total_keywords

total_keywords = uniqueKeyword(df["keywords"])
list(total_keywords[:200])

In [ ]:
common200 = total_keywords[:20]
mostCommonKey = [row[0] for row in common200[1:]] # seleziono le prime

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(50,30), dpi=100, facecolor='w', edgecolor='k')

# Creo una lista boleana 
def filter_df(initial_column, key):
    return key in initial_column

def correlationDataFrame(keywords, df, stringKeywordsColumnName):
    matrix = []
    for index, key in enumerate(keywords):
        row = [0] * len(keywords)
        boolean = df[stringKeywordsColumnName].apply(lambda x: filter_df(x, key))
        filtered_df = df[boolean]
        super_lista = []
        for lista in filtered_df[stringKeywordsColumnName]:
            super_lista.extend(lista)
        for indice, chiave in enumerate(keywords):
            #print('index = {}, conto = {}, chiave = {}'.format(indice, super_lista.count(chiave), chiave))
            row[indice] = super_lista.count(chiave)
        #normalizzo la righa, quindi il grafico non potrá essere letto per colonne
        valoremassimo = max(row)
        normalizzata = [x / valoremassimo for x in row]
        matrix.append(normalizzata)
        
    corrdf = pd.DataFrame(np.triu(matrix), columns=keywords, index=keywords)
    
    return  corrdf

corrdf = correlationDataFrame(mostCommonKey, df, 'keywords')
corrdf.to_csv('correlation.csv')


plt.pcolor(corrdf)
plt.yticks(np.arange(0.5, len(corrdf.index), 1), corrdf.index)
plt.xticks(np.arange(0.5, len(corrdf.columns), 1), corrdf.columns)
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
corrdf

In [ ]:
lista = []
for index, row in corrdf.iterrows():
    print(row.apply(lambda x: x if x>0.3 else None).to_string())
        

In [ ]:
print(lista)

l'analisi argomentativa ha portato a capire che nessuna keyword é particolarmente corelata. Pensiamo che sulle prime 200 é stata trovata solo una correlazione che superasse lo 0.4, che comunque é una correlazione debole.